<b>ToDO:</b>
- figure out how to recenter the predicted landmarks to actually match the image
- adjust the manga_dataset class to not require a landmarks .txt file
- adjust the loop in this script to not pull a landmarks file

In [1]:
import os
import sys
sys.path.append(os.path.dirname(sys.path[0]))
import numpy as np
import torch
import torch.nn as nn
import csv
from utils import *
from transformers import BeitForImageClassification
from tqdm import tqdm
from PIL import Image

from manga_dataset import MangaLandmarksDataset
from transform import TransformsNoAugmentation
from collections import OrderedDict
from transformers import BeitForImageClassification
from tqdm import tqdm
from PIL import Image


In [2]:
def postprocessing(pred, imgsz:int=24):
    """
    ret `pred` as the parameters then
    1. +0.5 for all preds-points and * imgsz(224) to get the exact position on the image
    2. .view(-1, 60, 2) to get the x-y coordinates for all landmarks on manga faces
    3. .squeeze() -> reducing array preds dimension
    4. converting to the numpy()
    """
    # print(pred.logits)
    # pred = (pred.cpu() + 0.5) * 224
    pred = (pred + 0.5) * imgsz
    pred = pred.view(-1,60,2)
    pred = pred.squeeze()
    return pred.detach().numpy()

def dataset_prepoessing(landmarks, img_size=224):
    """
    dataset_preprocessing() -> 
        1. +0.5 for all landmarks, due to -0.5 when loading dataset
        2. * imgs to get the exact position of that particular points on the image
    """
    landmarks_noa = (landmarks + 0.5) * img_size
    return landmarks_noa



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'beit'
weights_ls = ['microsoft/beit-base-patch16-224', 
    'microsoft/beit-base-patch16-384',
    'microsoft/beit-large-patch16-224',
    'microsoft/beit-large-patch16-384']
selected_weight = 0
variation = weights_ls[selected_weight].split('/')[-1]
network = BeitForImageClassification.from_pretrained(weights_ls[selected_weight], ignore_mismatched_sizes=True)
network.classifier = nn.Linear(in_features=network.classifier.in_features, out_features=120, bias=True)
# LOAD WEIGHTS INTO MODELS
# interested_epoch = 500
# weights_path = f'weights/{model_name}_224_{interested_epoch}.pth'
weights_path = 'weights/beit_rgb.pth'
network.load_state_dict(torch.load(weights_path))
network.to(device)
print(f'Device: {device}')
# print(f'Load weight@ {weights_path}')

# CREATE DIRECTORY FOR EVALUATION
create_directory_if_not_exists('eval')
save_model_name = weights_ls[selected_weight].split("/")[-1].replace("-", "_")
evaluation_name = f'{save_model_name}_eval'
create_directory_if_not_exists(f'eval/{evaluation_name}')

network.eval()

Device: cuda
Directory 'eval' already exists.
Directory 'eval/beit_base_patch16_224_eval' already exists.


BeitForImageClassification(
  (beit): BeitModel(
    (embeddings): BeitEmbeddings(
      (patch_embeddings): BeitPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BeitEncoder(
      (layer): ModuleList(
        (0): BeitLayer(
          (attention): BeitAttention(
            (attention): BeitSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (relative_position_bias): BeitRelativePositionBias()
            )
            (output): BeitSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (int

In [5]:
dataset_folder = 'dataset'
imgsz = 384
test_dataset = MangaLandmarksDataset(TransformsNoAugmentation(), folder=dataset_folder, split_set='test', imgsz=imgsz)

In [6]:

data_folder = "data/" + dataset_folder

files = os.listdir(data_folder)

image_path = data_folder+ "/" + files[0]

print(image_path)

data/dataset/FMA_test.jpg


In [7]:
dataset_folder = 'dataset'
#imgsz = 384
imgsz = 224
test_dataset = MangaLandmarksDataset(TransformsNoAugmentation(), folder=dataset_folder, split_set='test', imgsz=imgsz)


In [8]:
test_dataset.image_filenames

['data/dataset\\test\\FMA\\labels\\test\\FMA\\FMA_test.jpg']

This next block will be the actual pipeline. See what is needed.

In [9]:
class_length = dict()
pred_ls = []
gt_ls = []
network.eval()

# list of groundtruth landmarks coordinations
#landmarks = []
#for image, target, filename in test_dataset:
#    landmarks.append(target)

with torch.no_grad():
    for image, filename in tqdm(test_dataset):
        image = image.to(device)
        pred = postprocessing(network(image.view(1, 3, imgsz, imgsz)).logits.cpu(), imgsz=imgsz)
        #landmark = dataset_prepoessing(landmarks=landmark, img_size=imgsz)
        pred_ls.extend([pred]) # these seem to be the landmark coordinates, as predicted by the model
        #gt_ls.extend([landmark]) # these are used for comparison to a ground-truth

pred_class = dict()
#gt_class = dict()
class_length = dict()
#gt_class['Overall'] = []
pred_class['Overall'] = []
class_length['Overall'] = 60
for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
    #gt_class[name] = []
    pred_class[name] = []
#for gt in gt_ls:
#    gt_class['Overall'].extend(gt)
for pred in pred_ls:
    pred_class['Overall'].extend(pred)



for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
    (j, k) = FACIAL_LANDMARKS_IDXS[name]
    class_length[name] = k-j
    for gt_each in gt_ls:
        gt_class[name].extend(gt_each[j:k])
    for pred_each in pred_ls:
        pred_class[name].extend(pred_each[j:k])

chin_normalized_ls = []
 
for i, coord in enumerate(gt_ls):
    chin_normalized_ls.append(euclidean_distance(coord[:17][0], coord[:17][-1]))


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.27it/s]


In [10]:
help(TransformsNoAugmentation)


Help on class TransformsNoAugmentation in module transform:

class TransformsNoAugmentation(builtins.object)
 |  Methods defined here:
 |  
 |  __call__(self, image)
 |      Call self as a function.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [11]:
filename

'data/dataset\\test\\FMA\\labels\\test\\FMA\\FMA_test.jpg'

In [12]:
imgsz

224

In [13]:
image_copy = cv2.imread(image_path)
image_copy = cv2.resize(image_copy, (384, 384))
shape = pred_ls
shape = np.array([[round(val+50) for val in ls] for ls in shape[0]])
output = visualize_facial_landmarks(image_copy,shape)

#cv2.imshow('image',output)
#cv2.waitKey(0)

In [14]:
#shape = pred_ls
 
#shape2 = [[int(val) for val in ls] for ls in shape[0]]

# create two copies of the input image -- one for the
# overlay and one for the final output image
colors=None
alpha=0.75


overlay = image_copy.copy()
output = image_copy.copy()
# if the colors list is None, initialize it with a unique
# color for each facial landmark region
if colors is None:
    colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
              (168, 100, 168), (158, 163, 32), (100, 60, 200), 
              (100, 60, 200), (100, 60, 200), (180, 42, 220)]
    
# loop over the facial landmark regions individually
for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
    # grab the (x, y)-coordinates associated with the
    # face landmark
    (j, k) = FACIAL_LANDMARKS_IDXS[name]
    pts = shape[j:k]
   # pts = pts*1.5
    # check if are supposed to draw the jawline
    if name == "ChinContour":
        # since the jawline is a non-enclosed facial region,
        # just draw lines between the (x, y)-coordinates
        for l in range(1, len(pts)):
            ptA = tuple(pts[l - 1])
            ptB = tuple(pts[l])
            cv2.line(overlay, ptA, ptB, colors[i], 2)
    # otherwise, compute the convex hull of the facial
    # landmark coordinates points and display it
    else:
        hull = cv2.convexHull(pts)
        cv2.drawContours(overlay, [hull], -1, colors[i], -1)

# apply the transparent overlay
cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

cv2.imshow('image',output)
cv2.waitKey(0)

13

In [15]:
pts

array([[163, 215],
       [164, 210],
       [169, 207],
       [182, 209],
       [184, 210],
       [179, 216],
       [179, 217],
       [174, 225],
       [167, 224],
       [161, 219]])

In [16]:
shape2 = [[round(val) for val in ls] for ls in shape]
shape2

[[86, 139],
 [92, 160],
 [109, 179],
 [109, 197],
 [121, 213],
 [133, 226],
 [149, 240],
 [161, 253],
 [177, 262],
 [196, 260],
 [210, 250],
 [224, 236],
 [235, 216],
 [245, 197],
 [257, 178],
 [268, 157],
 [270, 133],
 [72, 85],
 [83, 89],
 [96, 89],
 [116, 101],
 [130, 101],
 [168, 97],
 [193, 93],
 [219, 89],
 [242, 82],
 [263, 81],
 [92, 142],
 [99, 138],
 [111, 138],
 [123, 141],
 [132, 141],
 [136, 150],
 [121, 149],
 [112, 152],
 [101, 148],
 [93, 144],
 [198, 143],
 [214, 138],
 [229, 136],
 [243, 134],
 [253, 134],
 [251, 140],
 [242, 143],
 [230, 141],
 [213, 140],
 [195, 146],
 [120, 141],
 [223, 136],
 [157, 178],
 [163, 215],
 [164, 210],
 [169, 207],
 [182, 209],
 [184, 210],
 [179, 216],
 [179, 217],
 [174, 225],
 [167, 224],
 [161, 219]]

In [20]:
# This part just writes thedata to a file. Currently, one file per image/frame. 
# class_length = dict()
data = [
    ["Overall"],
    ["ChinContour"],
    ["RightEyeBrow"],
    ["LeftEyeBrow"],
    ["RightEye"],
    ["LeftEye"],
    ["RightPupil"],
    ["LeftPupil"],
    ["Nose"],
    ["Mouth"],
    ]

# needs to be adjusted when this is all put into a loop
filepath_short = files[0].split(".")[0]

csv_filename = os.path.join("./tracking_output/", f'{filepath_short}_xy.csv')

data = np.array(data)

# save image with overlay
cv2.imwrite(os.path.join("./tracking_output/", f'{filepath_short}_tracked.jpg'), output)

# save coordinate data
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    #writer.writerows(data[:,1:])
    writer.writerows([["x", "y"]])
    writer.writerows(shape2)